<a href="https://colab.research.google.com/github/Sangyups/PyTorchZeroToAll/blob/main/lec_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F

# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n')

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===============\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')
for epoch in range(1, 10):
        print('===========================\n')
        train(epoch)
        test()

Training MNIST Model on cuda


Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.298821
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.308176
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.317404
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.309333
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.289514
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.306673
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.299628
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.291984
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.303942
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.292329
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.300876
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.307260
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.289408
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.297637
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.296092
Train E